In [1]:
# Load all the libraries you need to update your dataset with the newest emails
import imaplib
import email
from email.header import decode_header
import webbrowser
import os
import re # I need this package to filter particular text-patterns in my emails. --> Note: it is already built-in!
from datetime import datetime # I need this package to give the updated(!) dataset a good name, e.g. with to "today's" date in it
import pandas as pd

In [2]:
# Load the existing file that I need (= 'jobs-url.csv')

testo = pd.read_csv(
    "~/data/job-urls.csv" 
) 
testo # check, if it worked? --> yes!


,Date,url,Job Title,Employer Company
0,2023-01-29 05:53:08+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Data Analyst / Data Scientist (80%),"Institut für Bildungsevaluation, Zurich"
1,2023-01-29 05:53:08+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Praktikant /-in im Detailhandel,"VOI Migros Partner, Zürich"
2,2023-01-29 05:53:08+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Praktikant:in 50 - 100 %,"ZHAW, Zürich"
3,2023-01-29 05:53:08+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,KV Praktikum | Administration | KV-Praktikanti...,"Fromberg GmbH, Zürich"
4,2023-01-29 05:53:08+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Praktikant:in Social Media,"Ringier Axel Springer Schweiz AG, Zürich"
...,...,...,...,...
10767,2021-11-01 07:04:45+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Praktikant:in Social Media und Community SRF News,"Schweizer Radio und Fernsehen, Zürich"
10768,2021-11-01 07:04:45+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Initial Training Academy (ATCO) Instructor: AC...,"skyguide, Dübendorf"
10769,2021-11-01 07:04:45+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Praktikant:in Social Media und Community SRF News,"Schweizer Radio und Fernsehen, Zürich"
10770,2021-11-01 07:04:45+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Aktuar oder Ökonom in der Versicherungsaufsich...,"Eidgenössische Finanzmarktaufsicht FINMA, Bern"


In [3]:
# Check the gap between the data in our dataset VS. the date "today" --> this gap corresponds to the values I need to update!
datetime.now()

datetime.datetime(2023, 1, 31, 8, 21, 36, 74354)

In [4]:
# Set-Up our Email-Crawler (in order to scrape the latest emails)

# account credentials
username = "your_email_here@example_mail.com" # <- Change this to your own email (I deleted my credentials, since you should NEVER post those onto Github!)
password = "PASSWORD_OF_YOUR_EMAIL" # <- # Change this to your own email-password (I deleted my credentials, since you should NEVER post those onto Github!)
# use your email provider's IMAP server, you can look for your provider's IMAP server on Google
# or check this page: https://www.systoolsgroup.com/imap/
# for hotmail, it's this:
imap_server = "imap-mail.outlook.com" # checkout this link for other email-servers: https://www.systoolsgroup.com/imap/

def clean(text):
    # clean text for creating a folder
    return "".join(c if c.isalnum() else "_" for c in text)

In [5]:
# Establish a connection to the Email-Server:
# !Important: you need an internet-connection to run this cell!!!

# create an IMAP4 class with SSL 
imap = imaplib.IMAP4_SSL(imap_server)
# authenticate
imap.login(username, password)

('OK', [b'LOGIN completed.'])

In [6]:
# Get the NEWEST Emails from the server (= the ones I currently don't have in my dataset)

status, messages = imap.select("Jobs") # select the email folder, we want to scrape

# To update our DF, we need to know how many additional emails came into our 'Jobs'-folder: 
todays_date = datetime.now() # what is today's date?
todays_date = pd.to_datetime(todays_date, infer_datetime_format=True, utc= None) # we need a 'datetime'-object
latest_date_in_df = pd.to_datetime(testo.Date[0], utc = None, infer_datetime_format=True) # next, select the latest date from our current DDs (that we want to update)
latest_date_in_df = latest_date_in_df.replace(tzinfo=None) # we need to remove the time-zone, otherwise we CANNOT calculate how many new days we need to scrape in our email...
dates_to_scrape = todays_date - latest_date_in_df # the number of days we 
N = int(str(dates_to_scrape).split()[0])# <-- THIS is Key to understand: with this, we get the correct UPDATED total of emails in my inbox! --> formula: "N" = "today's date" - "latest date in our dataset"
print("To update our DF, we need to scrape {0} additional Mails".format(N))

# "messages" will contain the total number of emails in the folder "Jobs"
messages = int(messages[0])
print("We see that we currently have {0} emails in the folder 'Jobs'".format(messages)) # check, if it worked? --> we see that we currently have 232 emails in the folder "Jobs"

To update our DF, we need to scrape 2 additional Mails
We see that we currently have 260 emails in the folder 'Jobs'


In [7]:
# Now, read the Emails that I currently miss in my DF (update it)


dict_for_df = dict() # this empty dictionary will be transformed into the final time series ds later...
dates_array = [] # this will be my column "Date" in my later ds
total_jobs_available_today_array = [] # will be my column "Total Jobs available Today" 
new_jobs_today_array = [] # will be my column "New Jobs published Today"

### For 2nd DF:
dict_for_df2 = dict()
urls_array = [] # will be a list of lists (with an index of 5 --> this is KEY, because we need it to have the same length as the dates, since the dates will be our "observations"-unit in the DF2!)
job_titles_array = [] # same as "urls_array"...
companies_array = [] # same as "urls_array"...

for i in range(messages, messages-N, -1): # Why do we do this weird "backward"-loop? --> we want to iterate from the top to the bottom 
    # Key: in order to go "forward in time" (starting in the distant past), we need to iterate by typing in the 
    # following parameters: `messages-N, messages+1` and leave out "-1"(!!) at the end as the 3rd // last argument...
    # Backwards would be: `range(messages, messages-N, -1)`
    # fetch the email message by ID
    print("Currently reading {0}-th Email...".format(i))
    res, msg = imap.fetch(str(i), "(RFC822)") # `RFC822` is a special format that we can use to fetch the emails from the server: https://www.rfc-editor.org/rfc/rfc822
    for response in msg:
        if isinstance(response, tuple):
            msg = email.message_from_bytes(response[1]) # parse the bytes returned by the `fetch()`-method to a proper "Message"-object
            subject, encoding = decode_header(msg["Subject"])[0] # decode the "subject" of the email-address to human-readable Unicode.
            if isinstance(subject, bytes): # if the "subject" is from the data-type "bytes", decode to str
                subject = subject.decode(encoding)
            From, encoding = decode_header(msg.get("From"))[0] # decode email-sender (= "From") of the email-address to human-readable Unicode.
            if isinstance(From, bytes): # if the sender (= "From") is from the data-type "bytes", decode to str
                From = From.decode(encoding)
            print("Subject:", subject)
            print("From:", From)
            print(msg['Date'])
            todays_email_date = msg['Date']
            dates_array.append(todays_email_date) # append the dates to my empty array
            # if the email message is "multipart":  for instance, an email message can contain the "text/html"-content AND "text/plain"-parts, e.g. it has the HTML and(!) plain text versions of the message.
            if msg.is_multipart():
                # iterate over email parts
                for part in msg.walk():
                    # extract content type of email
                    content_type = part.get_content_type()
                    content_disposition = str(part.get("Content-Disposition"))
                    try:
                        # get the email body
                        body = part.get_payload(decode=True).decode()
                        lines = body.split('\n')
                        job_titles = []
                        companies = []
                        for line in lines:
                            if "https://www.jobs.ch/de/stellenangebote/detail/" in line:
                                job_titles.append(lines[lines.index(line) - 2])
                                companies.append(lines[lines.index(line) - 1])
                        matches_plural = re.findall(r".*neue Jobs.*", body)
                        matches_singular = re.findall(r".*neuer Job.*", body)
                        matches = matches_singular + matches_plural # concatenate the 2 lists into 1 (bigger) list
                        new_jobs_per_day = list(map(lambda string: int(string[0:2]), matches)) # take only the first element of each string (= which is the "number" - currently given as a `string` - that we are interested in, in order to calculate the total number of new open job position that opened "today")
                        total_new_jobs_today = sum(new_jobs_per_day)
                        urls = re.findall(r"https://www.jobs.ch/de/stellenangebote/detail/\S+", body)
                        currently_open_job_positions = len(urls)
                        total_jobs_available_today_array.append(currently_open_job_positions) 
                        new_jobs_today_array.append(total_new_jobs_today)
                        #print(matches)
                        #print(new_jobs_per_day)
                        #print(currently_open_job_positions)
                        #print(total_new_jobs_today)
                        #print(urls)
                        urls_array.append(urls)
                        #print(len(urls))
                        #print(job_titles)
                        job_titles_array.append(job_titles)
                        #print(len(job_titles))
                        #print(companies)
                        companies_array.append(companies)
                        #print(len(companies))
                        #print(body)
                        #print(matches_singular)
                        #print(matches_plural)
                    except:
                        pass
                    #if content_type == "text/plain" and "attachment" not in content_disposition:
                        # print text/plain emails and skip attachments
                        #print(body)
                    if "attachment" in content_disposition: # vorher: `elif` (statt `if`!)
                        # download attachment
                        filename = part.get_filename()
                        if filename:
                            folder_name = clean(subject)
                            if not os.path.isdir(folder_name):
                                # make a folder for this email (named after the subject)
                                os.mkdir(folder_name)
                            filepath = os.path.join(folder_name, filename)
                            # download attachment and save it
                            open(filepath, "wb").write(part.get_payload(decode=True))
            else:
                # extract content type of email
                content_type = msg.get_content_type()
                # get the email body
                body = msg.get_payload(decode=True).decode()
                #if content_type == "text/plain":
                    # print only text email parts
                    #print(body)
            #if content_type == "text/html":
                # if it's HTML, create a new HTML file and open it in browser
                #folder_name = clean(subject)
                #if not os.path.isdir(folder_name):
                    # make a folder for this email (named after the subject)
                    #os.mkdir(folder_name)
                #filename = "index.html"
                #filepath = os.path.join(folder_name, filename)
                # write the file
                #open(filepath, "w").write(body)
                # open in the default browser
                #webbrowser.open(filepath)
            print("="*100)
dict_for_df['Date'] = dates_array
dict_for_df['Total of open Job-Positions up until today'] = total_jobs_available_today_array
dict_for_df['New Jobs published Today'] = new_jobs_today_array

### 2nd DF:
for (date, url, title, company) in zip(dates_array, urls_array,  job_titles_array, companies_array):
    dict_for_df2[date] = [url, title, company] # this will add new keys to the dictionary, where the 'dates' are the keys and a list (of sublists, containing the URLs, Job-Titles and Company-Names) as the values...


Currently reading 260-th Email...
Subject: 67 neue Stellenangebote gefunden
From: "jobs.ch Job-Alarm" <jobmail@jobs.ch>
Fri, 27 Jan 2023 06:23:07 +0000
Currently reading 259-th Email...
Subject: 41 neue Stellenangebote gefunden
From: "jobs.ch Job-Alarm" <jobmail@jobs.ch>
Tue, 31 Jan 2023 06:22:39 +0000


In [8]:
# Check if it worked?
print(len(urls_array))
print(len(job_titles_array))
print(len(companies_array))

2
2
2


In [9]:
# create the 2nd DF --> for this, we need to create 3 separate DFs first...

rows = []
for key, values in dict_for_df2.items():
    for value in values:
        row = {key: value}
        rows.append(row)

list_of_df = []
for i in rows:
    df2 = pd.DataFrame(i)
    list_of_df.append(df2)

# Problem in all the DFs? --> we have the Date as 1 column, instead of rows... --> Solution: transform the DF into "long-format"...
test = list_of_df[1]
test.stack().droplevel(level=0).to_frame().rename({0: 'Job Title'}, axis='columns') # Key: 'droplevel'-method, because I had an unnecessary multi-index...  

url_series = pd.Series()
#all_series = pd.Series()
# Transform all the DFs for the column 'URL':
for i in list_of_df[0::3]: # Key: we start with the 1st list-element of 'list_of_df' and then only iterate over every 3rd elements
    url_series = pd.concat([url_series, i.stack().droplevel(level=0)])
    
title_series = pd.Series()
# Transform all the DFs for the column 'URL':
for i in list_of_df[1::3]: # Key: we start with the 1st list-element of 'list_of_df' and then only iterate over every 3rd elements
    title_series = pd.concat([title_series, i.stack().droplevel(level=0)])
    
company_series = pd.Series()
# Transform all the DFs for the column 'URL':
for i in list_of_df[2::3]: # Key: we start with the 1st list-element of 'list_of_df' and then only iterate over every 3rd elements
    company_series = pd.concat([company_series, i.stack().droplevel(level=0)])
    
#for i in list_of_df[0::3]: # Key: we start with the 1st list-element of 'list_of_df' and then only iterate over every 3rd elements
#    all_series = pd.concat([all_series, i.stack().droplevel(level=0)])
#    for j in list_of_df[1::3]: # Key: we start with the 1st list-element of 'list_of_df' and then only iterate over every 3rd elements
#        all_series = pd.concat([all_series, j.stack().droplevel(level=0)])
#        for k in list_of_df[2::3]: # Key: we start with the 1st list-element of 'list_of_df' and then only iterate over every 3rd elements
#            all_series = pd.concat([all_series, k.stack().droplevel(level=0)])

url_df = url_series.to_frame().rename({0: 'url'}, axis='columns').reset_index().rename({'index': 'Date'}, axis='columns')
title_df = title_series.to_frame().rename({0: 'Job Title'}, axis='columns').reset_index().rename({'index': 'Date'}, axis='columns')
company_df = company_series.to_frame().rename({0: 'Employer Company'}, axis='columns').reset_index().rename({'index': 'Date'}, axis='columns')

url_df['Date'] = pd.to_datetime(url_df['Date'])
title_df['Date'] = pd.to_datetime(title_df['Date'])
company_df['Date'] = pd.to_datetime(company_df['Date'])

<ipython-input-9-41087c1ef644>:18: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  url_series = pd.Series()
<ipython-input-9-41087c1ef644>:24: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  title_series = pd.Series()
<ipython-input-9-41087c1ef644>:29: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  company_series = pd.Series()


In [10]:
# Now, we can truly create our 2nd DF --> Prepare to merge Merging...
noob = pd.merge(
    url_df,
    title_df,
    how="left",
    on=url_df.index, # Key: we merge via the Index, which are only numbers from 0-184 --> merging via index. Note that I needed to put the "Date" as a column (and NOT as an index!)
).drop(['key_0', 'Date_y'], axis = 1) # we only need one of the "Date"-Columns. Also, we can drop the automatically generated column "key_0", which is a leftover-column that gets generated, because we merged via the "index". 

# second merge, since we need to combine 3 DFs into 1 "big" DF --> we currently only merged 2 (out of 3) DFs, that's why we need this 2nd merging...
noob = pd.merge(
    noob,
    company_df,
    how="left",
    on=noob.index, # Key: Again, we merge via the Index, which are ALL our observations...
).drop(['key_0', 'Date_x'], axis = 1)#.set_index('Date') # same as first merging: we drop one of the "Date"-Columns, as well as the automatically generated "index"-column (= 'key_0')... 

df2 = noob.sort_values(by="Date", ascending = False)
df2 = df2.reindex(columns=['Date', 'url', 'Job Title', 'Employer Company']) # change the order of columns: i want "date"-col to be the first one...

df2['Job Title'] = [el.replace("\r", "") for el in df2['Job Title']] # we need to remove the "\r" behind all the elements of the col of "Job Title" & "Employer Company", otherwise saving the ds correctly will not be possible! xD
df2['Employer Company'] = [el.replace("\r", "") for el in df2['Employer Company']] # same as above

In [11]:
df2 = df2.reset_index().drop(['index'], axis = 1)
df2

,Date,url,Job Title,Employer Company
0,2023-01-31 06:22:39+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Pharmabranche: Einstieg in die Software-Entwic...,"ProPharma Systems AG, Wettingen AG"
1,2023-01-31 06:22:39+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Praktikum Social Media «Tages-Anzeiger»,"Tamedia AG, Zürich"
2,2023-01-31 06:22:39+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Praktikum Social Media «Tages-Anzeiger»,"Tamedia AG, Zürich"
3,2023-01-31 06:22:39+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Praktikant Personalrekrutierung (m/w) 100% [Re...,"Freestar-Informatik AG, Stadt Zürich"
4,2023-01-31 06:22:39+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Praktikant/in Jugend- und Familienberatung,"Kantonale Verwaltung Zürich, Winterthur"
...,...,...,...,...
97,2023-01-27 06:23:07+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,"Pflegepraktikant/in (50 - 100 %), befristet fü...","Stadt Winterthur, Winterthur"
98,2023-01-27 06:23:07+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Praktikant/in KV 100% (w/m/d),"Cargologic AG, Zürich Flughafen"
99,2023-01-27 06:23:07+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Trainee-Programm – Unabhängige Vermögensberatu...,"VZ VermögensZentrum, Horgen"
100,2023-01-27 06:23:07+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Internship Sense Texture,"Givaudan, Kemptthal"


In [12]:
df_update = df2.append(testo)
df_update # check: should (currently) have 10'415 + 75 = 10'490 rows? --> yes!

,Date,url,Job Title,Employer Company
0,2023-01-31 06:22:39+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Pharmabranche: Einstieg in die Software-Entwic...,"ProPharma Systems AG, Wettingen AG"
1,2023-01-31 06:22:39+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Praktikum Social Media «Tages-Anzeiger»,"Tamedia AG, Zürich"
2,2023-01-31 06:22:39+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Praktikum Social Media «Tages-Anzeiger»,"Tamedia AG, Zürich"
3,2023-01-31 06:22:39+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Praktikant Personalrekrutierung (m/w) 100% [Re...,"Freestar-Informatik AG, Stadt Zürich"
4,2023-01-31 06:22:39+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Praktikant/in Jugend- und Familienberatung,"Kantonale Verwaltung Zürich, Winterthur"
...,...,...,...,...
10767,2021-11-01 07:04:45+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Praktikant:in Social Media und Community SRF News,"Schweizer Radio und Fernsehen, Zürich"
10768,2021-11-01 07:04:45+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Initial Training Academy (ATCO) Instructor: AC...,"skyguide, Dübendorf"
10769,2021-11-01 07:04:45+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Praktikant:in Social Media und Community SRF News,"Schweizer Radio und Fernsehen, Zürich"
10770,2021-11-01 07:04:45+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Aktuar oder Ökonom in der Versicherungsaufsich...,"Eidgenössische Finanzmarktaufsicht FINMA, Bern"


In [105]:
print(type(df_update))

<class 'pandas.core.frame.DataFrame'>


In [106]:
# Save the updated dataset
today = datetime.now().strftime('%Y-%m-%d') # we need to convert it into a string (in order to do string concatenation)
df_update.to_csv(f"../data/job-urls.csv", index=False)
df_update = df_update.reset_index().drop(['index'], axis = 1) # we need to reset the index after we stacked the two 
# DFs together --> if you don't do this, you CANNOT save it as a `.json`-object...
df_update.to_json(f"../data/job-urls-{today}.json")

In [107]:
df_update

,Date,url,Job Title,Employer Company
0,2023-01-29 05:53:08+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Data Analyst / Data Scientist (80%),"Institut für Bildungsevaluation, Zurich"
1,2023-01-29 05:53:08+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Praktikant /-in im Detailhandel,"VOI Migros Partner, Zürich"
2,2023-01-29 05:53:08+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Praktikant:in Product Management RED,"Ringier Sports, Zürich"
3,2023-01-29 05:53:08+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Praktikantinnen/Praktikanten Tagesstätten oder...,"Schweizerische Epilepsie-Stiftung, Zürich"
4,2023-01-29 05:53:08+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Praktikant:in 50 - 100 %,"ZHAW, Zürich"
...,...,...,...,...
10750,2021-11-01 07:04:45+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Praktikant:in Social Media und Community SRF News,"Schweizer Radio und Fernsehen, Zürich"
10751,2021-11-01 07:04:45+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Initial Training Academy (ATCO) Instructor: AC...,"skyguide, Dübendorf"
10752,2021-11-01 07:04:45+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Praktikant:in Social Media und Community SRF News,"Schweizer Radio und Fernsehen, Zürich"
10753,2021-11-01 07:04:45+00:00,https://www.jobs.ch/de/stellenangebote/detail/...,Aktuar oder Ökonom in der Versicherungsaufsich...,"Eidgenössische Finanzmarktaufsicht FINMA, Bern"


In [108]:
# close the connection and logout
imap.close()
imap.logout()

('BYE', [b'Microsoft Exchange Server IMAP4 server signing off.'])